# Saturação do **GRADIENTE**

Geralmente ocorre em funções de ativação não lineares como Sigmoid e Tangente Hiperbólica (tanh), acontece quando os gradientes se tornam muito pequenos, dificultando o aprendizado da rede

✅ Para evitar saturação:

- Prefira ReLU, Leaky ReLU ou ELU em vez de sigmoide/tanh.

- Use Batch Normalization, que mantém os valores das ativações dentro de uma faixa saudável.

# Fuga do **GRADIENTE**

Ocorre quando os gradientes se tornam muito grandes, fazendo com que os pesos explodam e a rede fique instável. Esse problema é comum em redes muito profundas, principalmente com funções de ativação que não possuem limite superior. pode acontecer em **Redes Recorrentes**, **Variantes da ReLU (Leaky ReLU, PReLU, ELU)**, funções que não possuem **limite superior para os valores da ativação**.

✅ Para evitar fuga do gradiente:

- Use Inicialização de Pesos adequada (ex: He Initialization para ReLU).

- Utilize Batch Normalization para manter os gradientes sob controle.

- Gradiente Clipping: Limita o tamanho do gradiente para evitar explosão.

- Regularização L2 (Weight Decay): Penaliza pesos muito grandes.

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [4]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# LeakyRelu

Utilizando a função de ativação *RelU* (onde o valor de entrada positivo permanece o mesmo, enquanto uma entrada negativa se torna zero), pode ser que o nosso modelo mate alguns neurônios pois se um neurônio recebe valores negativos continuamente, sempre retornará zero. A *LeakyRelu* visa consertar isso. Em sua função, ao invés de zerar valores de entrada negativa, ela os transforma em um fator α pequeno, geralmente α = 0.01, fazendo assim que em algum momento aquele neurônio "volte a vida", aprendendo novamente

## 📌 Como escolher a função de ativação da última camada?

| **Tipo de Problema**                                          | **Função de Ativação na Saída**         | **Explicação** |
|--------------------------------------------------------------|---------------------------------|--------------|
| **Classificação binária** (2 classes: "sim/não", "gato/cachorro") | `sigmoid`                     | Retorna um valor entre **0 e 1**, interpretado como **probabilidade**. |
| **Classificação multiclasse** (3 ou mais classes, previsão de uma única classe) | `softmax`                     | Converte os logits em **probabilidades normalizadas** (somam 1), útil para **1 classe por amostra**. |
| **Classificação multirrótulo** (cada amostra pode ter várias classes) | `sigmoid`                      | Cada saída pode ser ativada independentemente, pois cada rótulo é tratado separadamente. |
| **Regressão** (previsão de valores contínuos, como temperatura, preço de casa) | `linear` (ou nenhuma ativação) | Permite qualquer valor real como saída, sem restrições. |
| **Regressão para valores positivos** (exemplo: previsão de contagem) | `relu` ou `softplus`          | Garante que a saída seja **sempre positiva**. |


In [6]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), # Tamanho das imagens
    keras.layers.Dense(300, kernel_initializer='he_normal'),
    keras.layers.LeakyReLU(alpha=0.2), # pode ser "PReLU()" no lugar da LeakyReLU, mas se quiser usar SELU precisa definir "activation='selu'" e "kernel_initializer='cun_normal'"
    keras.layers.Dense(100, kernel_initializer='he_normal'),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Dense(10, activation='softmax') # 10 pois são 10 classes possíveis de saída. (classes das fotos do MNIST)
])

c:\Users\anton\OneDrive\Desktop\RedesNeurais-Basico\environment\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\anton\OneDrive\Desktop\RedesNeurais-Basico\environment\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [9]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-3), metrics=['accuracy'])

In [10]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.4985 - loss: 1.6399 - val_accuracy: 0.7214 - val_loss: 0.8766
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7320 - loss: 0.8300 - val_accuracy: 0.7750 - val_loss: 0.7001
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7741 - loss: 0.6908 - val_accuracy: 0.8008 - val_loss: 0.6255
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7918 - loss: 0.6270 - val_accuracy: 0.8134 - val_loss: 0.5816
Epoch 5/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8037 - loss: 0.5867 - val_accuracy: 0.8194 - val_loss: 0.5498
Epoch 6/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8117 - loss: 0.5566 - val_accuracy: 0.8236 - val_loss: 0.5298
Epoch 7/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8198 - loss: 0.5366 - val_accuracy: 0.8326 - val_loss: 0.5085
Epoch 8/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8199 - loss: 0.5282 - 

# ❌ Piora no modelo

Por que será que o modelo utilizando o leakyRelU piorou em relação ao que fizemos no arquivo "RedesSequenciais.ipynb" utilizando uma rede sequencial simples?



Fator |	Modelo com LeakyReLU (pior desempenho) |	Modelo com ReLU (melhor desempenho)
|------------------------------------------------------|---------------------------------|--------------|
Função de ativação |	LeakyReLU(α=0.2) (mantém gradientes negativos, pode atrapalhar) |	ReLU (aprendizado mais rápido e eficiente)
Inicialização |	he_normal (pode gerar pesos grandes demais) |	glorot_uniform (mais equilibrado para redes pequenas)
Complexidade |	Mais camadas e inicializações desnecessárias |	Modelo mais simples e eficiente

# 🎲 batch Normalization

Utilizada para **normalizar** as **ativações das camadas intermediárias** de uma rede neural, o **BathNormalization** acelera o treinamento, melhora a performance e reduz a sensibilidade a inicialização dos pesos.

Quando treinamos uma rede neural, as ativações das camadas podem ficar em diferentes escalas, o que pode dificultar a cnovergência durante o treinamento, já que o gradiente pode se tornar muito grande ou muito pequeno. Isso é conhecido como **covariate shift** (Mudança na distribuição das ativações das camadas). O **BatchNormalization** garante que as ativações tenham distribuição com média **0** e desvio padrão **1** durante o treinamento.

In [13]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

c:\Users\anton\OneDrive\Desktop\RedesNeurais-Basico\environment\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-3) ,metrics=['accuracy'])

In [15]:
model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5779 - loss: 1.2571 - val_accuracy: 0.7992 - val_loss: 0.5991
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7826 - loss: 0.6293 - val_accuracy: 0.8222 - val_loss: 0.5197
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8041 - loss: 0.5611 - val_accuracy: 0.8398 - val_loss: 0.4831
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8225 - loss: 0.5148 - val_accuracy: 0.8478 - val_loss: 0.4597
Epoch 5/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8283 - loss: 0.4919 - val_accuracy: 0.8510 - val_loss: 0.4432
Epoch 6/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8347 - loss: 0.4786 - val_accuracy: 0.8538 - val_loss: 0.4326
Epoch 7/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8389 - loss: 0.4596 - val_accuracy: 0.8580 - val_loss: 0.4213
Epoch 8/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8375 - loss: 0.4600 - 